In [ ]:
import duckdb

parquet_path = "data/merged_output_parquet/*.parquet"

First we test that we can read our parquet files with DuckDB by checking that all columns are present.

In [ ]:
duckdb.read_parquet(f"{parquet_path}").columns

We do some initial data anlysis by calculating the Average, Minimum, and Maximum values for the most interesting columns.

Also we calculate the speed of execution se we can later compare it to Dask SQL.

We see that we should do some filtering because minimum for trip_distance and total_amount is negative, which are not results that should be possible. Also we should cap the total_amount because we doubt that anyone would pay 1000003.8 for a taxi ride.

In [ ]:
%%time

duckdb.sql(f""" select 
                avg(passenger_count) avg_passengers, 
                avg(trip_distance),
                min(trip_distance),
                max(trip_distance),
                avg(total_amount),
                min(total_amount),
                max(total_amount) 
           from '{parquet_path}'
""")

Calculate the median trip distance of rides that were done for each day of the year, we separete the results by month so we can visualize the results.

In [ ]:
%%time

result = duckdb.sql(f""" select month(tpep_pickup_datetime), day(tpep_pickup_datetime), round(mean(trip_distance), 2) as avg_distance from  
           '{parquet_path}'
           group by day(tpep_pickup_datetime), month(tpep_pickup_datetime) 
           order by month(tpep_pickup_datetime), day(tpep_pickup_datetime)
""").fetchall()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker

We are going to visualize the results we have calculated earlier, so we get a better understanding for the trips taken during the year. As we can see, for the first four months of the year, people are usually taking shorter trips compared to the other parts of the year. 

In [ ]:
df = pd.DataFrame(result, columns=['month', 'day', 'avg_distance'])
df = df.sort_values(by=['month', 'day'])
df['date'] = df.apply(lambda row: str(int(row['day'])) + "-" + str(int(row['month'])), axis=1)


plt.figure(figsize=(18, 7))

plt.bar(df['date'], df['avg_distance'], label='Daily Avg Distance')

plt.title('Average Trip Distance Over the Year', fontsize=18)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Trip Distance (miles)', fontsize=14) 

plt.xticks(rotation=45)

ax = plt.gca()
# ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.xaxis.set_major_locator(ticker.FixedLocator(df[df['day'] == 1].index))
# ax.axis.set_major_locator( df[df['day'] == 1]['date']) 

# Adding a grid for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Adding a legend
plt.legend(fontsize=12)

# Show the plot
plt.tight_layout() # Adjusts plot to ensure everything fits without overlapping
plt.show()

Next we take a look at the median fare amount during different hours of the day. As we can see the taxi drivers earn the most in the early morning, during 4, 5, 6, hours in the morning. Also there are some inconsistencies which should be looked at, we have a negative median fare amount for the 8th hour of the day.

In [ ]:
result =  duckdb.sql(f""" select hour(tpep_pickup_datetime), round(mean(fare_amount), 2) as avg_amount from  
           '{parquet_path}'
           group by hour(tpep_pickup_datetime)
           order by hour(tpep_pickup_datetime)
""").fetchall()

df = pd.DataFrame(result, columns=['hour', 'fare_amount'])
df = df.sort_values(by=['hour'])

plt.figure(figsize=(18, 7))

plt.bar(df['hour'], df['fare_amount'], label='Hourly Average Fare Amount')

plt.title('Hourly Average Fare Amount', fontsize=18)
plt.xlabel('Hour', fontsize=14)
plt.ylabel('Average Fare Amount', fontsize=14) 

plt.xticks(rotation=45)

ax = plt.gca()
# ax.xaxis.set_major_locator(ticker.MaxNLocator(12))
ax.xaxis.set_major_locator(ticker.FixedLocator(df['hour'].index))
# ax.axis.set_major_locator( df[df['day'] == 1]['date']) 

# Adding a grid for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Adding a legend
plt.legend(fontsize=12)

# Show the plot
plt.tight_layout() # Adjusts plot to ensure everything fits without overlapping
plt.show()

We take a look at the borughs that people usually take trips between. And also calculate the time needed to calculate this query in order to compare it to the Dask SQL.

In [ ]:
%%time

duckdb.sql(f""" select borough_pickup, borough_dropoff, count(*) as trips_count from  
           '{parquet_path}'
           where borough_pickup is not null and borough_dropoff is not null
           group by borough_pickup, borough_dropoff
           order by trips_count desc
""").fetchall()

In the following cells, we try to make some conclusion based on the data we have added to the original data. We take the look at the fact if the day is rainy or if it is a holiday.

In [ ]:
duckdb.sql(f""" select is_holiday, round(avg(trip_distance), 2), round(mean(tip_amount), 2) median_tip from  
           '{parquet_path}'
           group by is_holiday 
""")

In [ ]:
duckdb.sql(f""" select round(avg(no_trips)) avg_trips_holiday from
           (
                select year(tpep_pickup_datetime), month(tpep_pickup_datetime), day(tpep_pickup_datetime), count(*) as no_trips from  
                '{parquet_path}'
                where is_holiday is true
                group by year(tpep_pickup_datetime), month(tpep_pickup_datetime), day(tpep_pickup_datetime)
           )
""")

In [ ]:
duckdb.sql(f""" select round(avg(no_trips)) as avg_trips_non_holiday from
           (
                select year(tpep_pickup_datetime), month(tpep_pickup_datetime), day(tpep_pickup_datetime), round(count(*)) as no_trips from  
                '{parquet_path}'
                where is_holiday is false
                group by year(tpep_pickup_datetime), month(tpep_pickup_datetime), day(tpep_pickup_datetime)
           )
""")

In [ ]:
duckdb.sql(f""" select round(avg(trip_distance), 2), round(mean(tip_amount), 2) median_tip from  
           '{parquet_path}'
           where "rain (mm)" >= 0.5
""")

In [ ]:
duckdb.sql(f""" select round(avg(trip_distance), 2), round(mean(tip_amount), 2) median_tip from  
           '{parquet_path}'
           where "rain (mm)" < 0.5
""")

We are creating an Dask client in order to compute the queries using the Dask SQL.

In [ ]:
from dask.distributed import Client, 
from dask_sql import Context

client = Client(n_workers=4, threads_per_worker=1, memory_limit='8GB')
print(f"Dask Dashboard link: {client.dashboard_link}")

In [ ]:
parquet_path = "data/merged_output_parquet" 
c = Context()


dask_table_name = "taxi_data"
c.create_table(dask_table_name, parquet_path, format="parquet")


sql_query = f""" select borough_pickup, borough_dropoff, count(*) as trips_count from  
           '{dask_table_name}'
           where borough_pickup is not null and borough_dropoff is not null
           group by borough_pickup, borough_dropoff
           order by trips_count desc
"""

sql_query_initial = f"""
    select 
                avg(passenger_count) avg_passengers, 
                avg(trip_distance),
                min(trip_distance),
                max(trip_distance),
                avg(total_amount),
                min(total_amount),
                max(total_amount) 
           from '{dask_table_name}'
"""


For this initial query we can see that by using DuckDB we can save a lot of time compared to Dask SQL implementation. 
For my local implementation I can notice that using DuckDB needs 2.11 ms to compute this query, where as Dask SQL requires 6.48 s.

In [ ]:
%%time 
c.sql(sql_query_initial).compute()

The same thing can be noticed in this example here, DuckDB requires significently less time to calculate the same query comapred to Dask SQL. 
We can see that the Dask SQL locally takes 46.4 s to compute this query, while DuckDB need only 669 ms, which is more than 50 times faster.

In [ ]:
%%time

c.sql(sql_query).compute()

## Comparison

In [2]:
#!/usr/bin/env python
import os, time, glob, contextlib, textwrap
import duckdb, pandas as pd, dask.dataframe as dd, dask
from dask.diagnostics import ProgressBar
from dask_sql import Context

# ------------------------------------------------------------------
# paths
# ------------------------------------------------------------------
BASE          = "/Users/amadej/Desktop/big_data/assignment5/big-data-project/data/sample_merged_output"
PARQUET_GLOB  = f"{BASE}/part*.parquet"
PARQUET       = sorted(glob.glob(PARQUET_GLOB))

CSV_DIR       = f"{BASE}/csv_cache"
os.makedirs(CSV_DIR, exist_ok=True)
CSV_GLOB      = f"{CSV_DIR}/part*.csv"

# one‑time parquet to csv export (so benchmarking is fair)
for p in PARQUET:
    stem = os.path.basename(p)[:-8]
    out  = f"{CSV_DIR}/{stem}.csv"
    if not os.path.exists(out):
        pd.read_parquet(p).to_csv(out, index=False)
CSV = sorted(glob.glob(CSV_GLOB))

# columns actually used by the three test queries
USECOLS = [
    "tpep_pickup_datetime",
    "trip_distance",
    "tip_amount",
    "borough_pickup",
    "borough_dropoff",
]

# ------------------------------------------------------------------
# helper utilities
# ------------------------------------------------------------------
results = []                       # wall‑times end up here
def timer(label, qid, fmt):
    """context‑manager that appends duration to results list"""
    @contextlib.contextmanager
    def _t():
        t0 = time.perf_counter();  yield
        results.append(dict(engine=label, query=qid, fmt=fmt,
                            sec=time.perf_counter() - t0))
    return _t()

# three toy analytics
def q1(df):               # average trip distance
    return df["trip_distance"].mean()

def q2(df):               # 10 busiest borough OD pairs
    return (
        df.groupby(["borough_pickup", "borough_dropoff"])
          .size()
          .nlargest(10)
    )

def q3(df):               # mean tip by hour‑of‑day
    df2 = df.assign(hour = dd.to_datetime(df["tpep_pickup_datetime"]).dt.hour
                    if isinstance(df, dd.DataFrame)
                    else pd.to_datetime(df["tpep_pickup_datetime"]).dt.hour)
    return df2.groupby("hour")["tip_amount"].mean()

QUERIES = {"Q1": q1, "Q2": q2, "Q3": q3}
SQL = {
    "Q1": "SELECT AVG(trip_distance) FROM df",
    "Q2": textwrap.dedent("""
         SELECT borough_pickup, borough_dropoff, COUNT(*) trips
         FROM df
         GROUP BY 1,2
         ORDER BY trips DESC
         LIMIT 10
    """),
    "Q3": textwrap.dedent("""
         WITH tmp AS (
           SELECT
             date_part('hour', CAST(tpep_pickup_datetime AS TIMESTAMP)) AS hr,
             tip_amount
           FROM df
         )
         SELECT
           hr,
           AVG(tip_amount) AS avg_tip
         FROM tmp
         GROUP BY hr
    """),
}


# ------------------------------------------------------------------
# global Dask memory limits & spilling
# ------------------------------------------------------------------
dask.config.set({
    "distributed.worker.memory.target": 0.60,
    "distributed.worker.memory.spill" : 0.70,
    "distributed.worker.memory.pause" : 0.85,
})

# ------------------------------------------------------------------
# benchmark loop parquet vs csv
# ------------------------------------------------------------------
for fmt, paths, glob_pat in [
        ("parquet", PARQUET, PARQUET_GLOB),
        ("csv",     CSV,     CSV_GLOB)]:

    # ------------- DuckDB ---------------------------------------------------
    con = duckdb.connect()
    con.execute(f"CREATE OR REPLACE VIEW df AS SELECT * FROM read_{fmt}('{glob_pat}')")
    for qid in QUERIES:
        with timer("duckdb", qid, fmt):
            con.execute(SQL[qid]).fetchall()

    # ------------- Pandas ----------------------------------------------------
    if fmt == "parquet":
        pdf = pd.concat([pd.read_parquet(p, columns=USECOLS) for p in paths])
    else:
        pdf = pd.concat([
            pd.read_csv(
                p,
                low_memory=False,
                usecols=USECOLS,
                parse_dates=["tpep_pickup_datetime"],
                dtype={"borough_pickup":"string",
                       "borough_dropoff":"string"},
            ) for p in paths
        ])
    for qid, fn in QUERIES.items():
        with timer("pandas", qid, fmt):
            fn(pdf)

    # ------------- Dask DataFrame -------------------------------------------
    if fmt == "parquet":
        ddf = dd.read_parquet(
            paths,
            columns=USECOLS,
            gather_statistics=False,
            blocksize="16MB",
            split_row_groups=True,
            engine="pyarrow",
        )
    else:
        # tell Dask that tpep_pickup_datetime is datetime64
        ddf = dd.read_csv(
            paths,
            usecols=USECOLS,
            assume_missing=True,
            dtype_backend="pyarrow",
            blocksize="16MB",
        )

    ddf["tpep_pickup_datetime"] = dd.to_datetime(ddf["tpep_pickup_datetime"])

    for qid, fn in QUERIES.items():
        with ProgressBar(), timer("dask", qid, fmt):
            fn(ddf).compute()

    # ------------- Dask‑SQL --------------------------------------------------
    ctx = Context()
    ctx.create_table("df", PARQUET_GLOB)   # <-- a single string
    for qid in QUERIES:
        with ProgressBar(), timer("dask-sql", qid, fmt):
            ctx.sql(SQL[qid]).compute()


# ------------------------------------------------------------------
# pretty print
# ------------------------------------------------------------------
tbl = pd.DataFrame(results).pivot_table(
        index=["engine", "fmt"], columns="query", values="sec")
print("\nWall‑time (seconds) on three sample files:\n")
print(tbl.round(3))


# Summary:
#
# Wall-time (seconds):
#                Q1      Q2      Q3
# dask     csv   4.922   5.194   7.056
#          parquet 0.110   2.336   0.230
# dask-sql csv   0.241   2.591   0.681
#          parquet 0.234   2.832   0.525
# duckdb   csv   1.076   1.123   1.138
#          parquet 0.012   0.035   0.041
# pandas   csv   0.007   0.673   0.378
#          parquet 0.015   1.034   0.655
#
# Key takeaways:
# 1) Parquet versus CSV:
#    - Dask and DuckDB see huge wins with Parquet over CSV.
#    - Pandas still reads CSV slightly faster for simple scans, but group-bys tip the balance toward Parquet.
#    - Dask-SQL sits between: it benefits from Parquet but has extra SQL-layer cost.
#
# 2) Comparing engines on Parquet:
#    - DuckDB is unbeatable on Parquet (Q1 in 0.012 s, Q2 in 0.035 s, Q3 in 0.041 s).
#    - Dask (pure) is next: parallelism cuts Q3 to 0.230 s.
#    - Dask-SQL adds flexibility but pays a premium (Q2 takes 2.832 s vs. 2.336 s in raw Dask).
#    - Pandas holds its own on simple queries (Q1 0.015 s) but slows on group-bys (Q2 1.034 s).
#
# 3) Comparing engines on CSV:
#    - Pandas excels at Q1 on CSV (0.007 s) and remains competitive for simple group-bys.
#    - DuckDB on CSV is consistent (~1.1 s across all queries).
#    - Dask-SQL on CSV (Q1 0.241 s) outperforms DuckDB on Q1, but for grouping (Q2/Q3) it’s slower.
#    - Pure Dask on CSV is the slowest (5–7 s range), due to parsing and scheduling overhead.
#
# Conclusion:
# - For quick ad-hoc lookups on Parquet, DuckDB is the clear winner.
# - For scale-out or larger-than-memory workloads on Parquet, raw Dask is a strong candidate.
# - Dask-SQL offers SQL syntax over Dask but with measurable overhead—best when you need that integration.
# - Pandas remains great for small CSV analyses, but switch to columnar formats and DuckDB or Dask when queries get heavier.


[########################################] | 100% Completed | 104.61 ms
[########################################] | 100% Completed | 2.35 sms
[########################################] | 100% Completed | 204.93 ms
[########################################] | 100% Completed | 205.54 ms
[########################################] | 100% Completed | 2.24 ss
[########################################] | 100% Completed | 101.79 ms
[########################################] | 100% Completed | 105.82 ms
[########################################] | 100% Completed | 624.05 ms
[########################################] | 100% Completed | 5.90 ss
[########################################] | 100% Completed | 7.60 ss
[########################################] | 100% Completed | 10.14 s
[########################################] | 100% Completed | 310.92 ms
[########################################] | 100% Completed | 2.26 ss
[########################################] | 100% Completed | 102.91 ms
[##